<a href="https://colab.research.google.com/github/yash-mandi/e3l.ai-model/blob/main/Memory_Context_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Install required libraries
!pip install -U langchain-community
!pip install langchain faiss-cpu sentence-transformers transformers accelerate ipywidgets bitsandbytes --quiet
!pip install git+https://github.com/huggingface/transformers.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 110.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 84.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 133.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 105.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.

In [ ]:
# Imports
import torch
import ipywidgets as widgets
from IPython.display import display, clear_output, Javascript
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
import ipywidgets as widgets
from IPython.display import display, clear_output
torch.set_float32_matmul_precision('high')

In [ ]:
# Load tokenizer and model
model_path = "/content/drive/MyDrive/Gemma-Merged-Finetune-Model"  # Update as needed
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto")

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    pad_token_id=tokenizer.eos_token_id
)

Device set to use cuda:0


In [ ]:
# ----------- Memory Setup -----------
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = None  # Global memory store

def add_to_memory(question, answer):
    """Add a Q&A pair to FAISS memory"""
    global vectorstore
    doc = Document(page_content=f"Q: {question}\nA: {answer}")
    if vectorstore is None:
        vectorstore = FAISS.from_documents([doc], embedding_model)
    else:
        vectorstore.add_documents([doc])

def get_memory_context(query, k=3):
    """Return top-k similar past Q&As as context"""
    if vectorstore is None:
        return ""
    results = vectorstore.similarity_search(query, k=k)
    return "\n\n".join([doc.page_content for doc in results])

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# 🧠 Memory Tracking
memory = []

# Detect continuation-type questions
def is_continuation(question):
    return question.lower().strip() in [
        "continue", "continue from where you left", "go on", "keep going"
    ]

def add_to_memory(question, answer):
    if not is_continuation(question):
        memory.append((question, answer))

def get_memory_context():
    context = []
    count = 0
    for q, a in reversed(memory):
        if is_continuation(q):
            continue
        context.append(f"Q: {q}\nA: {a}")
        count += 1
        if count == 10:
            break
    return "\n".join(reversed(context))

# 🧠 Prompt Builder
def build_prompt(user_input):
    memory_context = get_memory_context()
    prompt = (
        (memory_context + "\n\n") if memory_context else ""
    ) + (
        f"Question: {user_input}\n"
        "Answer:"
    )
    return prompt

In [ ]:
# ✅ Chat UI
def create_prompt_widget():
    label = widgets.HTML("<b>Ask your question:</b>")
    text_area = widgets.Textarea(
        placeholder='Enter your question here...',
        layout=widgets.Layout(width='90%', height='60px')
    )
    submit_button = widgets.Button(description="Submit", button_style='success')
    output_box = widgets.Output()
    container = widgets.VBox()

    def on_submit_clicked(b):
        output_box.clear_output()
        question = text_area.value.strip()
        if not question:
            with output_box:
                print("❗ Please enter a question.")
            return

        try:
            with output_box:
                print("Generating response... Please wait.")

            prompt = build_prompt(question)
            response = pipe(
                prompt,
                max_new_tokens=300,
                temperature=0.8,
                top_p=0.95,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id
            )

            generated_text = response[0]['generated_text']
            answer_start = generated_text.find("Answer:")
            generated_answer = (
                generated_text[answer_start + len("Answer:"):].strip()
                if answer_start != -1 else generated_text
            )

            with output_box:
                output_box.clear_output()
                print(f"Answer:\n{generated_answer}")

            add_to_memory(question, generated_answer)

            # Add a new input widget below after answer
            new_input = create_prompt_widget()
            container.children += (new_input,)

        except Exception as e:
            with output_box:
                output_box.clear_output()
                print(f"⚠️ Error generating response:\n{str(e)}")

    submit_button.on_click(on_submit_clicked)
    container.children = [label, text_area, submit_button, output_box]
    return container

# Display the first prompt
display(create_prompt_widget())